In [1]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque
import time

offene Probleme / mögliche Verbesserungen:
    - in einem Endzustand nicht mehr q(a',s') berechnen
    - Minibatch mit TF erzeugen
    - store old network weights and use them in loss function

In [15]:
# build graph
env = gym.make('LunarLander-v2')
# env = gym.make('CartPole-v0')

obs_dim = env.observation_space.shape[0]
n_acts = env.action_space.n
gamma = 0.9

obs_ph = tf.placeholder(shape=(None,obs_dim), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None), dtype=tf.int32)
rew_ph = tf.placeholder(shape=(None), dtype=tf.float32)
new_obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
terminal_ph = tf.placeholder(shape=(None), dtype=tf.float32)

# make core of state-action-value function network
mlp_action_val = tf.keras.models.Sequential()
mlp_action_val.add(tf.keras.layers.Dense(50, activation='relu'))
mlp_action_val.add(tf.keras.layers.Dense(50, activation='relu'))
mlp_action_val.add(tf.keras.layers.Dense(n_acts, activation=None))

# define state action values
old_state_action_values = mlp_action_val(obs_ph)
new_state_action_values = mlp_action_val(new_obs_ph)

# select action
greedy_action = tf.math.argmax(old_state_action_values, axis=1)

# define loss function
y = rew_ph + gamma * tf.reduce_max(new_state_action_values, axis=1)*(1-terminal_ph)
y_no_grad = tf.stop_gradient(y) 
action_masks = tf.one_hot(act_ph, n_acts)
old_selected_action_values = tf.reduce_sum(action_masks * old_state_action_values, axis=1)
# action_value_loss = tf.reduce_mean((y_no_grad - old_selected_action_values)**2)
action_value_loss = tf.losses.mean_squared_error(y_no_grad, old_selected_action_values)

# define optimizer
optimizer_action_value = tf.train.AdamOptimizer(0.01)
train_action_value = optimizer_action_value.minimize(action_value_loss)
gradients = optimizer_action_value.compute_gradients(action_value_loss)

In [16]:
def dqn(sess, env, experience_replay_buffer, n_episodes):
    row_pointer = 0
    learnable = False
    for i in range(n_episodes):
        done = False
        state = env.reset()
        cnt = 0
        while not done:
            action = select_action(sess, state, 0.1)
            new_state, reward, done, _ = env.step(action)
            add_sample(state, action, reward, new_state, done, row_pointer)
            row_pointer += 1
            if row_pointer == exp_replay_size:
                learnable = True
            row_pointer %= exp_replay_size
            state = new_state
            if learnable:
                update_q_function()
            cnt += 1
        print(cnt)
#         if i % 100 == 0:
#             print(evaluate(sess, env))

            
def add_sample(state, action, reward, new_state, done, row_pointer):
    experience_replay_buffer[row_pointer, :obs_dim] = state
    experience_replay_buffer[row_pointer, obs_dim:2*obs_dim] = new_state
    experience_replay_buffer[row_pointer, 2*obs_dim] = action
    experience_replay_buffer[row_pointer, 2*obs_dim+1] = reward
    experience_replay_buffer[row_pointer, 2*obs_dim+2] = float(done)
    
def evaluate(sess, env):
    sum_return = 0
    for i in range(100):
        done = False
        state = env.reset()
        while not done:
#             if i == 0:
#                 env.render()
            action = select_action(sess, state, 0)
            new_state, reward, done, _ = env.step(action)
            state = new_state
            sum_return += reward
        env.close()
    return sum_return / 100
            

        
def update_q_function():
    experience_batch = sample_experience_batch()
    states, actions, rewards, new_states, terminal_flags = extract_data(experience_batch)
    loss = gradient_step(states, actions, rewards, new_states, terminal_flags)

    
def gradient_step(states, actions, rewards, new_states, terminal_flags):
#     old_loss = sess.run([action_value_loss], feed_dict={
#                                 obs_ph: np.array(states).reshape(-1, obs_dim),
#                                 act_ph: np.array(actions),
#                                 rew_ph: np.array(rewards).reshape(-1,1),
#                                 new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
#                                 terminal_ph: np.array(terminal_flags).reshape(-1,1)
#                              })
#     print('Gradients:',sess.run(gradients, feed_dict={
#                                 obs_ph: np.array(states).reshape(-1, obs_dim),
#                                 act_ph: np.array(actions),
#                                 rew_ph: np.array(rewards),
#                                 new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
#                                 terminal_ph: np.array(terminal_flags)
#                              }))
    
    _, loss, debug_output = sess.run([train_action_value, action_value_loss, y], feed_dict={
                                obs_ph: np.array(states).reshape(-1, obs_dim),
                                act_ph: np.array(actions),
                                rew_ph: np.array(rewards),
                                new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
                                terminal_ph: np.array(terminal_flags)
                             })
#     new_loss = sess.run([action_value_loss], feed_dict={
#                                 obs_ph: np.array(states).reshape(-1, obs_dim),
#                                 act_ph: np.array(actions),
#                                 rew_ph: np.array(rewards).reshape(-1,1),
#                                 new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
#                                 terminal_ph: np.array(terminal_flags).reshape(-1,1)
#                              })
#     print('Old loss vs new loss', old_loss, new_loss)
#     old_state_value = sess.run([old_state_action_values], feed_dict={
#                                 obs_ph: np.array([0,0,0,0]).reshape(-1, obs_dim)
#                              })
#     print(old_state_value)
    return loss
    
def extract_data(experience_batch):
    states = [x[:obs_dim] for x in experience_batch]
    actions = [x[2*obs_dim] for x in experience_batch]
    rewards = [x[2*obs_dim+1] for x in experience_batch]
    new_states = [x[obs_dim:2*obs_dim] for x in experience_batch]
    terminal_flags = [x[2*obs_dim+2] for x in experience_batch]
    return states, actions, rewards, new_states, terminal_flags
    
def sample_experience_batch(batch_size=32):
    indices = np.random.choice(np.arange(len(experience_replay_buffer)), batch_size)
    sample_batch = experience_replay_buffer[indices]
    return sample_batch

def select_action(sess, state, eps):
    if np.random.rand() < eps:
        action = np.random.randint(n_acts)
    else:
        action = sess.run(greedy_action, {obs_ph: state.reshape(1,-1)})[0]
    return action

In [22]:
# main

exp_replay_size = 500
# experience_replay_buffer = deque([], exp_replay_size)
memory_width = 3 + 2*obs_dim
experience_replay_buffer = np.zeros((exp_replay_size, memory_width))
# env = gym.make('CartPole-v0')
# env = gym.make('FrozenLake-v0')
sess = tf.Session()
sess.run(tf.global_variables_initializer())
starttime = time.time()
dqn(sess, env, experience_replay_buffer, 100)
stoptime = time.time()
print(stoptime-starttime)

53
75
79
78
86
82
73
82
95
87
76
55
52
53
71
133
105
73
75
159
62
71
74
161
127
252
108
143
415
264
83
143
109
106
192
85
189
99
130
218
181
99
230
61
222
112
172
172
117
1000
146
97
146
112
486
1000
249
239
805
93
85
101
122
103
99
133
93
105
96
321
68
246
194
79
161
286
150
128
185
111
66
135
214
158
129
200
171
109
147
149
490
428
155
395
104
105
64
61
93
71
62.597747802734375
